In [8]:
#keeping this jsut for reference.

,created_utc,ups,link_id,id,author,body,parent_id,root,depth,num_siblings,num_children,num_comments_author,num_previous_comments,num_later_comments,time_since_root,time_since_parent,num_comments_subtree,height_subtree
0,1430438400,3.0,34f9rh,cqug90j,jesse9o3,No one has a European accent either because i...,cqug2sr,cqug2sr,1,1,0,1,0,0,0.000000,0.000000,1,0
1,1430438400,3.0,34fvry,cqug90k,beltfedshooter,That the kid ..reminds me of Kevin. so sad :-(,34fvry,34fvry,1,1443,0,1,0,3512,0.000000,0.000000,1,0
2,1430438400,5.0,34ffo5,cqug90z,InterimFatGuy,NSFL,cqu80zb,cqu80zb,1,5,0,1,0,11,0.000000,0.000000,1,0
3,1430438401,1.0,34aqsn,cqug91c,JuanTutrego,I'm a guy and I had no idea this was a thing g...,cqtdj4m,cqtdj4m,1,4,0,1,0,4,0.000000,0.000000,1,0
4,1430438401,101.0,34f9rh,cqug91e,dcblackbelt,"Mid twenties male rocking skinny jeans/pants, ...",cquc4rc,cquc4rc,1,2,1,1,0,3,0.000000,0.000000,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4234965,1433116795,NaN,37y5rx,crrbelu,monster860,Does the sun set in the west/rise in the east?...,37y5rx,37y5rx,1,1496,0,1,3265,0,12.452222,0.000000,1,0
4234966,1433116795,NaN,37y5rx,crrbelv,jsimo36,Coffee.,37y5rx,37y5rx,1,1496,0,1,3265,0,12.452222,0.000000,1,0
4234967,1433116796,NaN,380jx2,crrbemp,torianicole731,"people who cannot make up their mind, my bulls...",380jx2,380jx2,1,9,0,1,8,0,0.522500,0.000000,1,0
4234968,1433116797,NaN,37yawp,crrbenh,bellypouch,Give them to Irish people in exchange for doin...,37yawp,37yawp,1,1779,0,1,3519,0,11.398056,0.000000,1,0


### **Toulouse School of Economics**
#### **M2 Statistics & Econometrics**
---
## **Web Mining**
### Final Project
---
#### Anh-Dung LE
#### Sherman ALINE
---

## Import necessary packages and setting for multiprocessing 

In [201]:
import pandas as pd
import numpy as np
import string, unidecode, re, os, json
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms.traversal.depth_first_search import dfs_tree
#from dask.distributed import Client
#import dask.dataframe as dd
from nltk.corpus import stopwords
from multiprocessing import dummy

#core = os.cpu_count() - 2
#client = Client(n_workers = core, threads_per_worker = 1, memory_limit = '25GB')
my_stopwords = stopwords.words('english')
#P = dummy.Pool(processes = core)

## Import dataset

In [59]:
from pathlib import Path
path = Path('/Users/sma/Desktop/LEARNING/M2/S2/PROJ/webmin/comments_students.csv')
df = pd.read_csv(path, header = 0, engine = 'c',
                 usecols = ['created_utc', 'ups', 'link_id', 'id', 'author', 'body', 'parent_id'],
                nrows = 10000000)

In [60]:
df.head()

,created_utc,ups,link_id,id,author,body,parent_id
0,1430438400,3.0,t3_34f9rh,cqug90j,jesse9o3,No one has a European accent either because i...,t1_cqug2sr
1,1430438400,3.0,t3_34fvry,cqug90k,beltfedshooter,That the kid ..reminds me of Kevin. so sad :-(,t3_34fvry
2,1430438400,5.0,t3_34ffo5,cqug90z,InterimFatGuy,NSFL,t1_cqu80zb
3,1430438401,1.0,t3_34aqsn,cqug91c,JuanTutrego,I'm a guy and I had no idea this was a thing g...,t1_cqtdj4m
4,1430438401,101.0,t3_34f9rh,cqug91e,dcblackbelt,"Mid twenties male rocking skinny jeans/pants, ...",t1_cquc4rc


We remove prefix so that columns `parent_id` and `link_id` have the same format as column `id`.

In [61]:
df['parent_id'] = df['parent_id'].str.split('_', expand = True)[1]
df['link_id'] = df['link_id'].str.split('_', expand = True)[1]

`ind` is the indices of  "normal" rows that will be fed into the model later.

In [62]:
ind = (df['body'] != 'deleted') & (df['body'] != '[deleted]') & df['body'].notna()

We save this dataset for later use.

In [ ]:
#THIS PART IS WHERE DUNGS CODES FOR GRAPHS ARE. DELETED FOR EASE OF DEVELOPMENT.
#path = r'E:\\M2 EconStat\\Web Mining\\prefix_removed.csv'
#df.to_csv(path, index = False, encoding =  'utf-8')

Let's summerize $10$ graph features we've extracted so far.

- `depth`:

- `num_siblings`:

- `num_children`:

- `num_comments_author`:

- `num_previous_comments`:

- `num_later_comments`:

- `time_since_root`:

- `time_since_parent`:

- `num_comments_subtree`:

- `height_subtree`:

## Text featurization

As a user of reddit I will apply some domain knowledge in the process of selecting text features. 

* text emotes - 
* emojis - emoticons may be used to detect sentiment. additionally they may signal a overly-casual post which may not be recieved well.

* sentiment in post - funny comments, as well as serious and informative posts will get upvotes. in long comment threads, a hostile tone may get upvotes (for one party, while the other party with hostile tone will get many downvotes)
    * (using TF-IDF) ?
    * BERT ?

* length - long comments may be an indication of high-information or high-effort post

* #punctuation - this is another indicator that a post is likely to be well-written, correlated with useful information and understandable writing etc. all things which will get upvotes on AskReddit. Extremely large amounts of punctuation may also indicate visual text jokes being made using markdown.

* #paragraphs - indicator of high amiount of content/contribution, high effort. 
               - we detect paragraphs by the numbe rof double spaces. doublespaces are removed on reddit, and thourhg manual investigation I have determined that double spaces in our corpora are new lines.

* capital letters

* number of links in a post - citations generally get upvotes, an indicator of sharing information.

* sensitive content: contains 'NSFW' or 'NSFL'

* refer to other users /u/*

* refer to subreditt /r/

* quotes - detect , count number of them, remove.

markdown: https://daringfireball.net/projects/markdown/ https://www.reddit.com/r/reddit.com/comments/6ewgt/reddit_markdown_primer_or_how_do_you_do_all_that/


### special cases to ignore;
* AutoModerator comments
* NA upvotes amount...

In [ ]:
#unparse html entities
#+ get number of links & remove them
# - replace quotes with [[QUOTE]], (quotes are of the form ) '>[text]  
#-remove rows where body is NA.
#- extract emojis, extract emoticons, remove them
'
#- get length, number of punctuation, capital letters, num paragraphs (by finding \n characters)
#- run text cleaning
#- sentiment analysis shit.

# - &gt; is > which is used for quotes.

In [ ]:
import HTMLParser
# TODO &lt; &gt; probably others...

# '   - ' bullet points (with a newline but that always works that way.)
# '  ' new line.

In [72]:
#df_text = df.dropna(subset=['body'])



In [192]:
df_text.loc[df.body.str.contains('/r/', na=False)]
#df_text.loc[df_text.id=='crqm50x']

,created_utc,ups,link_id,id,author,body,parent_id
74,1430438429,4.0,34ffo5,cqug9mj,TatTater,If they had just chosen to spawn in a first wo...,cqu5ixs
75,1430438429,1.0,lnjol,cqug9mm,AutoModerator,**PLEASE READ THIS MESSAGE IN ITS ENTIRETY BEF...,lnjol
106,1430438440,1.0,34gc1q,cqug9va,Puplis,/r/ems brings a bunch of cold hot sandwiches f...,34gc1q
188,1430438468,4.0,34f9rh,cqugafn,escalat0r,"This is actually pretty funny, the only place ...",cqu8epb
353,1430438540,1.0,34gc1q,cqugbvl,konaborne,/r/firstworldanarchists brings a single sandwi...,34gc1q
...,...,...,...,...,...,...,...
4234768,1433116663,NaN,380np2,crrbc7k,AutoModerator,**Attention!** Please keep in mind that the OP...,380np2
4234775,1433116666,NaN,37yoz6,crrbc9x,Cafrilly,/r/parahumans,crqurfg
4234796,1433116678,NaN,380nq7,crrbchu,AutoModerator,**PLEASE READ THIS MESSAGE IN ITS ENTIRETY BEF...,380nq7
4234870,1433116729,NaN,380ntu,crrbdf3,AutoModerator,**Attention!** Please keep in mind that the OP...,380ntu


In [ ]:
#ಠ

In [86]:
import marko 

In [157]:
marko.convert("I quit   What?!    [You have no idea, how high I can fly](https://www.youtube.com/watch?v=VYCehJk7UDE)    That scene popped up when you said >and get repeatedly overlooked by corporate despite performing well...    Michael seems to get the shit end of stick even though he's just a good guy, a bit of a goof, but still just a genuine good guy")

'<p>I quit   What?!    <a href="https://www.youtube.com/watch?v=VYCehJk7UDE">You have no idea, how high I can fly</a>    That scene popped up when you said &gt;and get repeatedly overlooked by corporate despite performing well...    Michael seems to get the shit end of stick even though he\'s just a good guy, a bit of a goof, but still just a genuine good guy</p>\n'

In [147]:
dir(temp)
#temp

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_prefix',
 'children',
 'inline_children',
 'link_ref_defs',
 'match',
 'override',
 'parse',
 'parse_inline',
 'priority',
 'virtual']

In [156]:
#len(temp.children)
temp.override

False

In [ ]:

#emoji.

## Build Features

In [268]:
#define some strings used in formatting...
newline = '  '
bullet = '   - '
lessthan = '&lt;'
greaterthan = '&gt;'

In [ ]:


## var: # markdown links
#df['md_link_count'] = df.body.str.count('\[(?P<txt>[^\]]+)\]\((?P<lnk>[^\)]+)\)')
#167563.0

## move markdown links to a new variable. if we want to do something with them later we can...
#link_data = df.body.str.extractall('\[(?P<txt>[^\]]+)\]\((?P<lnk>[^\)]+)\)')
##regex explanation: find object of form [txt](lnk) where txt doesnt contain ], lnk doenst contain )

## remove them from the text.
#df['body'] = df.body.str.replace('\[(?P<txt>[^\]]+)\]\((?P<lnk>[^\)]+)\)', '\g<txt>')

#pasted links with no markdown
#df['nomk_link_count'] df.body.str.count('(http|https)://')) 

##reddit links: count then remove the /r/ part but keep the name of sub.
#df['reddit_links'] = df.body.str.count('/r/') #works
#df['body'] = df.body.str.replace('/r/','')

#user links 
#df['reddit_links'] = df.body.str.count('/u/') #works
#df['body'] = df.body.str.replace('/u/','')

#count quotes and remove them.
df['num_quotes'] = df.body.str.count(greaterthan+'(.+)'+newline)
df.body.str.replace(greaterthan+'(.+)'+newline,'')

0          No one has a European accent either  because i...
1           That the kid ..reminds me of Kevin.   so sad :-(
2                                                       NSFL
3          I'm a guy and I had no idea this was a thing g...
4          Mid twenties male rocking skinny jeans/pants, ...
                                 ...                        
4234965    Does the sun set in the west/rise in the east?...
4234966                                             Coffee. 
4234967    people who cannot make up their mind, my bulls...
4234968    Give them to Irish people in exchange for doin...
4234969                                 classy way to put it
Name: body, Length: 4234970, dtype: object

In [263]:
link_data

txt  \
        match                                                      
27      0                                        Here's the link   
43      0                                           those idiots   
75      0               contact the moderators of this subreddit   
131     0      I'm working on filling in the blanks in the me...   
196     0                         he did a better job than Obama   
...                                                          ...   
4234870 0      any replies that are jokes, puns, off-topic, o...   
        1               contact the moderators of this subreddit   
4234927 0                                               flynning   
4234936 0      any replies that are jokes, puns, off-topic, o...   
        1               contact the moderators of this subreddit   

                                                             lnk  
        match                                                     
27      0      http://en.wikipedia.org/wiki/Rick_James#Legal_...  
43      0              https://m.youtube.com/watch?v=wptn5RE2I-k  
75      0                      /message/compose/?to=/r/AskReddit  
131     0                   http://lawcomic.net/guide/?page_id=5  
196     0      http://www.forbes.com/sites/briandomitrovic/20...  
...                                                          ...  
4234870 0      http://www.reddit.com/r/AskReddit/wiki/index#w...  
        1                      /message/compose/?to=/r/AskReddit  
4234927 0      http://tvtropes.org/pmwiki/pmwiki.php/Main/Fly...  
4234936 0      http://www.reddit.com/r/AskReddit/wiki/index#w...  
        1                      /message/compose/?to=/r/AskReddit  

[167563 rows x 2 columns]

In [261]:
link_data.txt[4234936,0]

"Serious] replies only** tag, therefore [any replies that are jokes, puns, off-topic, or are otherwise non-contributory will be removed](http://www.reddit.com/r/AskReddit/wiki/index#wiki_--.5Bserious.5D_tags--).  If you don't fall within the scope the question is directed to, please do not reply to the question as your comment will be removed. If the question is Bakers of reddit... and you're not a baker, your comment will be removed. All top-level replies need to be from someone who is in the group the question was asked to.  If you see others posting comments that violate this tag, please report them to the mods!   Thanks for your cooperation and enjoy the discussion!   *I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit"

In [210]:
df.body[76764]

"It's funny you say this because I like learning the theoretical aspect of things without learning the math (because, as this thread is talking about, fuck math).  I don't really know the math behind quantum mechanics, but I understand the claims and ideas it makes (mostly).  It's funny because I learned from the top down.  I looked at quantum mechanics and I was like what the hell is this? and then I looked okay, it deals with the way quantum particles interact.  Well how do they interact? and then I looked at okay, electrons can be in two places at once, wait, how?  Like basically I went to the hardest stuff and then followed the rabbit trails back down to the beginning.  As a result, I never really had to unlearn anything.  Now granted, I'm talking about learning broad concepts for fun and I've probably got a lot of misinformation because I wasn't formally educated, but I think that makes education so much more interesting.  Math was so boring until I asked my teacher if there was a

## Text cleaning

In [23]:
# TODO: write text cleaning function which actually works.

#def text_cleaning(df, colname, ind):
#    
#    tmp = df[colname][ind].copy()
#    
#    # Convert text to lowercase
#    tmp = tmp.str.lower()
#    
#    # Delete punctuation
#    tmp = tmp.str.replace('\n', ' ')
#    tmp = tmp.str.replace('\r', ' ')    
#    tmp = tmp.str.replace(r"((?!{}).)".format('(\\b[-/]\\b|[a-zA-Z0-9])'), ' ', regex = True)
#    
#    # Tokenize
#    tmp = tmp.str.split()
#    
#    # Delete stop words
#    tmp = tmp.apply(lambda x: [w for w in x if w not in my_stopwords])
#    
#    # Reverse tokenize
#    df.loc[ind, colname] = tmp.map(lambda word: ' '.join(word))
#    
#
#df2 = text_cleaning(df, 'body', ind)

## TF-IDF

In [24]:
df.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [15]:
data = df['body']
y = df['ups']

n_feature = 50

tfi_df_vec = TfidfVectorizer(use_idf = True,
                             max_features = n_feature)

X = tfi_df_vec.fit_transform(data)

TypeError: 'NoneType' object is not subscriptable

## Split dataset into test and train sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.33,
                                                    random_state = 42)

## Random forest model

In [ ]:
model = RandomForestRegressor(n_jobs = -1)
model.fit(X_train, y_train)

# Get the mean absolute error on the validation data
y_pred = model.predict(X_test)
MAE = mean_absolute_error(y_test , y_pred)
print('Random forest validation MAE = ', MAE)

## XGBoost model

In [ ]:
# from xgboost import XGBRegressor
# XGBModel = XGBRegressor()
# XGBModel.fit(X_train, y_train , verbose = False)

# # Get the mean absolute error on the validation data :
# XGBpredictions = XGBModel.predict(X_test)
# MAE = mean_absolute_error(y_test , XGBpredictions)
# print('XGBoost validation MAE = ', MAE)

## Preliminary deep neural network

In [ ]:
# model = Sequential()
# model.add(Dense(1000, input_dim = 1000, activation = 'relu', kernel_initializer='normal'))
# model.add(Dense(8, activation = 'relu', kernel_initializer='normal'))
# model.add(Dense(1, activation = 'linear', kernel_initializer='normal'))
# model.compile(loss = 'mean_absolute_error',
#               optimizer = 'adam',
#               metrics = ['accuracy'])
# print(model.summary())
# model.fit(X_train, y_train,
#           epochs = 3,
#           batch_size = 10,
#           validation_data = (X_test, y_test),
#           verbose = 1)